## Compilation



In [1]:
cd $HOME/models/lif_cpp
make clean
make

| lto-wrapper: | warning: | using      | serial | compilation | of   | 2 | LTRANS | jobs |
| [            | Babel    | evaluation | exited | with        | code | 0 | ]      |      |

## Notebook Settings



    %load_ext autoreload
    %autoreload 2
    %reload_ext autoreload
    
    %run /home/leon/models/lif_cpp/notebooks/setup.py
    %matplotlib inline
    %config InlineBackend.figure_format = 'png'

    The autoreload extension is already loaded. To reload it, use:
      %reload_ext autoreload
    Python exe
    /home/leon/mambaforge/envs/dual_data/bin/python

:END:



## Imports



    import warnings
    # Ignore FutureWarning
    warnings.simplefilter(action='ignore', category=FutureWarning)
    
    import sys
    sys.path.insert(0, '/home/leon/models/lif_cpp')  
    REPO_PATH = "/home/leon/models/lif_cpp"
    
    import subprocess
    import pandas as pd
    from time import sleep
    from yaml import safe_load
    from scipy.stats import circmean
    from run_model import run_cpp, update_conf, restore_conf
    from analysis.decode import decode_bump, circcvl
    
    pal = [sns.color_palette("tab10")[0], sns.color_palette("tab10")[1]]



## Helpers



    from joblib import Parallel, delayed
    import numpy as np
    
    def work(name, X_str, ini, phi, X):
        try:
            session = "/%s_%s_%.3f_phi_%d_ini_%d/" % (name, X_str, X, phi, ini)
            rate = get_data(session)
            return rate[:32000, -1]
        except:
            return None
    
    def get_rates_X_parallel(name, X_list, ini_list, phi_list, X_str='Jee', n_jobs=-1):
        # Create a parallel instance using n_jobs
        parallel = Parallel(n_jobs=n_jobs)
    
        # Use delayed to set up a list of tasks
        tasks = (delayed(work)(name, X_str, ini, phi, X) for ini in ini_list for phi in phi_list for X in X_list)
    
        # Execute the tasks in parallel
        rates_data = parallel(tasks)
    
        # Removing None values resulted from exceptions and build the rates array
        # rates_data = list(filter(None, rates_data))
        rates = np.array(rates_data)
        print(rates.shape)
        rates = rates.reshape(len(ini_list), len(phi_list), len(X_list), 32000, -1)
        return rates

    def get_data(session='/'):
      # Open the file in binary mode and read the data
      with open('/home/leon/models/lif_cpp/data/simul/'+ session + '/rates.txt', 'rb') as f:
        data = np.fromfile(f, dtype=np.float32)
      # Now 'data' is a numpy array containing your data
      return data.reshape(-1, 40000).T

    def get_rates_X(name, X_list, ini_list, phi_list, X_str='Jee'):
       rates = []
       for ini in ini_list:
          for phi in phi_list:
             for X in X_list:
                try:
                   session = "/%s_%s_%.2f_phi_%d_ini_%d/" % (name, X_str, X, phi, ini)
                   rate = get_data(session)
                   rates.append(rate[:32000])
                except:
                   pass
    
       rates = np.array(rates)
       rates = rates.reshape(len(ini_list), len(phi_list), len(X_list), 32000, -1)
       return rates

    def get_df_ini_X(rates, X_list, X_str='Jee'):
        n_trials, n_phi, n_X, n_neurons, n_times = rates.shape
    
        # Create indices
        # trials_ind, phi_ind, X_ind, neurons_ind, times_ind = np.indices((n_trials, n_phi, n_X, n_neurons, n_times))
    
        trials_ind, phi_ind, X_ind, neurons_ind, times_ind = np.meshgrid(
            np.arange(n_trials),
            np.arange(n_phi),
            np.arange(n_X),
            np.arange(n_neurons),
            np.arange(n_times),
            indexing='ij'
        )
    
        # Construct DataFrame
        df = pd.DataFrame({
            'trial': trials_ind.flatten(),
            'phi': phi_ind.flatten(),
            X_str : X_ind.flatten() * np.round((X_list[-1] - X_list[0]) / (X_list.shape[0] -1), 5) + X_list[0],
            'neuron': neurons_ind.flatten(),
            'time': times_ind.flatten() * 0.1,
            'rates': rates.flatten()
        })
    
        return df

    def load_data(name, X_list, ini_list, phi_list, X_str='Jee'):
        rates = get_rates_X_parallel(name, X_list, ini_list, phi_list, X_str)
        print(rates.shape)
        df = get_df_ini_X(rates, X_list, X_str)
        return df

    def get_df_code(df, X_str='Jee'):
        df_code = df.groupby(['time', 'trial', 'phi', X_str])['rates'].apply(decode_bump).reset_index()
        df_code[['m0', 'm1', 'phase']] = pd.DataFrame(df_code['rates'].tolist(), index=df_code.index)
        df_code = df_code.drop(columns=['rates'])
    
        end_point = df_code[df_code.time==df_code.time.iloc[-1]]      
        end_point = end_point.drop(columns=['time'])
        print(end_point.head())  
        return df_code, end_point 

    def run_loop_X(conf_name, name, X_str, X_list, ini_list, phi_list, axis=0):
        restore_conf(REPO_PATH + '/conf/' + conf_name + '.yml',
                     REPO_PATH + '/conf/' + conf_name + '.yml.bak')
    
        conf_path = REPO_PATH + '/conf/'+ conf_name + '.yml'
    
        for ini in ini_list:
            for phi in phi_list:
                for X in X_list:
                    session = "%s_%s_%.3f_phi_%d_ini_%d" % (name, X_str, X, phi, ini)
                    data_path = REPO_PATH + '/data/simul/' + session
                    update_conf(REPO_PATH + '/conf/'+ conf_name, 'DATA_PATH', data_path)
                    update_conf(REPO_PATH + '/conf/' + conf_name, X_str, float(X), axis=axis)
                    update_conf(REPO_PATH + '/conf/' + conf_name, 'PHI_STIM', float(phi), axis=0)
                    update_conf(REPO_PATH + '/conf/' + conf_name, 'IF_REC_SPIKE', 0)
                    update_conf(REPO_PATH + '/conf/' + conf_name, 'T_SAVE', 1000)
    
                    sleep(.1)
                    run_cpp(session, conf_path=conf_path)
                    sleep(.1)
    
                    subprocess.run([REPO_PATH + '/src/mem_usage.sh'])
                    subprocess.run([REPO_PATH + '/src/cpu_usage.sh'])
    
        restore_conf(REPO_PATH + '/conf/' + conf_name + '.yml.bak',
                     REPO_PATH + '/conf/' + conf_name + '.yml')

    def get_precision(x):
    
        cmean =  (x - circmean(x, low=-np.pi, high=np.pi)) 
    
        cmean[cmean > np.pi] -= 2*np.pi
        cmean[cmean < -np.pi] += 2*np.pi
    
        return cmean

    import scipy.stats as stats
    
    def plot_smooth(data, ax, color):
        mean = data.mean(axis=0)  
        ci = smooth.std(axis=0, ddof=1) * 1.96
    
        # Plot
        ax.plot(mean, color=color)
        ax.fill_between(range(data.shape[1]), mean - ci, mean + ci, alpha=0.25, color=color)



## Phase Space



### Varying Ie



#### Parameters



    name = 'odr'
    Ie_list = np.linspace(2*.85, 2*1.15, 11)
    
    conf_name = 'config'
    name = 'last'
    # name = 'bistability'
    name = 'bistability'
    
    # Ie_list = np.linspace(1.6, 2.0, 11)
    # Ie_list = np.linspace(1.6, 2.2, 11)
    print(Ie_list)
    
    ini_list = np.arange(0, 10)
    print(ini_list)
    
    phi_list = np.linspace(0, 360, 9)[1:]
    print(phi_list)
    
    # Ie_list=[3]
    # ini_list = [0]
    phi_list = [0, 180]

    [1.7  1.76 1.82 1.88 1.94 2.   2.06 2.12 2.18 2.24 2.3 ]
    [0 1 2 3 4 5 6 7 8 9]
    [ 45.  90. 135. 180. 225. 270. 315. 360.]

    n_sim = len(Ie_list) * len(ini_list) * len(phi_list) 
    total_seconds = n_sim * .46
    hours = total_seconds // 3600  # number of hours
    total_seconds %= 3600  # remaining seconds after hours are accounted for
    
    minutes = total_seconds // 60  # number of minutes
    seconds = total_seconds % 60  # remaining seconds after minutes are accounted for
    print('n_sim', n_sim, 'Expected runtime', f"{hours}h {minutes}m {seconds}s")

    n_sim 880 Expected runtime 0.0h 6.0m 44.80000000000001s



#### Simulation



    config = safe_load(open("/home/leon/models/lif_cpp/conf/config_single.yml", "r"))
    Iext = np.array(config['Iext'])  
    Jab = np.array(config['Jab'])
    
    BALANCE = ((Iext[0] / Jab[0]) / (Iext[1] / Jab[2])) > ((Jab[1] / Jab[0]) / (Jab[3] / Jab[2]))
    print('Balance', BALANCE)
    
    BALANCE = (Iext[0] / Iext[1]) > (Jab[1] / Jab[3])
    print(BALANCE)
    
    rates = -np.dot(np.linalg.inv(Jab.reshape((2 ,2))), Iext)
    print('rates', rates)
    
    J = Jab[1] * Jab[2] / (Jab[0] * Jab[3])
    Q = (Jab[3] * Jab[1] * Iext[0] - Iext[1]) / Jab[2]
    
    print('stp G', J)
    print('stp Q', Q)

    Balance True
    True
    rates [-0.12445964  0.89273148]
    stp G 0.10160765895953758
    stp Q 1.754

    run_loop_X(conf_name, name, 'Iext', Ie_list, ini_list, phi_list, axis=0)

    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...



#### Analysis



###### Load Data



    df = load_data(name, Ie_list, ini_list, phi_list, 'Iext')
    df['Iext'] = df['Iext'].round(5)
    df_code, end_point = get_df_code(df, 'Iext')
    
    # end_point['accuracy'] = (end_point.phase - end_point['phi'] / 180 * np.pi) % (2 * np.pi)
    end_point['precision'] = end_point.groupby(['phi', 'Iext'], group_keys=False)['phase'].apply(get_precision) * 180 / np.pi
    
    df_smooth = df[df.time==df.time.iloc[-1]].groupby(['time', 'trial', 'phi', 'Iext'])['rates'].apply(circcvl).reset_index()

    (220, 32000)
    (10, 2, 11, 32000, 1)
       trial  phi  Iext        m0        m1     phase
    0      0    0  1.70  0.186875  0.097323 -0.393267
    1      0    0  1.76  0.232250  0.033491 -0.798763
    2      0    0  1.82  0.321750  0.118784 -2.584054
    3      0    0  1.88  0.410500  0.100168 -1.823707
    4      0    0  1.94  0.515625  0.098280 -0.018989

    idx_off = np.round(Ie_list[3], 5)
    idx_on = np.round(Ie_list[7], 5)
    print('parameters', idx_off, idx_on)

    parameters 1.88 2.12

    smooth=[]
    point = df_smooth[df_smooth.Iext==idx_off].reset_index()  
    for i in range(point.rates.shape[0]):
        m0, m1, phase = decode_bump(point.rates[i])
        smooth.append(np.roll(point.rates[i], int(( phase / 2.0 / np.pi - 0.5) * point.rates[i].shape[0])))
    
    smooth = np.array(smooth)
    
    smooth_on = []
    point_on = df_smooth[df_smooth.Iext==idx_on].reset_index()  
    for i in range(point_on.rates.shape[0]):
        m0, m1, phase = decode_bump(point_on.rates[i])
        smooth_on.append(np.roll(point_on.rates[i], int((phase / 2.0 / np.pi - 0.5) * point_on.rates[i].shape[0])))
    smooth_on = np.array(smooth_on)



###### tuning



    fig, ax = plt.subplots(1, 2, figsize=[2.*width, height])
    
    categories = df_code[['trial', 'phi']].drop_duplicates()
    colors = plt.cm.jet(np.linspace(0, 1, len(categories)))
    for (trial, phi) in categories.values:
      # Select the subset of the data corresponding to the current trial and phi
      subset = df_code[(df_code['trial'] == trial) & (df_code['phi'] == phi)]
    
      # Plot the phase vs time for the current trial and phi
      ax[0].plot(subset['Iext'], subset['m0'], alpha=.25, marker='o', lw=0)
      ax[1].plot(subset['Iext'], subset['m1'], alpha=.25, marker='o', lw=0)
    
    ax[0].set_xlabel('FF Input')
    ax[0].set_ylabel('$\mathcal{F}_0$ (Hz)')
    ax[1].set_ylabel('$\mathcal{F}_1$ (Hz)')
    ax[1].set_xlabel('FF Input (Hz)')
    
    plt.savefig(name + '_tuning.svg', dpi=300)

![img](./.ob-jupyter/a6ca396b3acaf004387e673cc2ffc5b3c1467609.png)

    0.008*1.675

    0.0134

    N_E = 32000
    
    df_point = end_point[end_point.Iext==idx_off]
    df_point_on = end_point[end_point.Iext==idx_on]
    
    fig, ax = plt.subplots(1, 3, figsize=[2.25*width, height])
    
    sns.lineplot(end_point, x='Iext', y=end_point['m0'], ax=ax[0], legend=False, marker='o', lw=0, hue='phi')
    ax[0].set_xlabel('FF Input')
    ax[0].set_ylabel('$\mathcal{F}_0$ (Hz)')
    ax[0].set_ylim([0., 10])
    
    # sns.lineplot(end_point, x='Iext', y=end_point['m1'], ax=ax[1], legend=False, marker='o', color='k')
    
    sns.lineplot(end_point, x='Iext', y=end_point['m1']/end_point['m0'], ax=ax[1], legend=False, marker='o', color='k', lw=0, hue='trial')
    sns.lineplot(end_point, x=idx_off, y=df_point['m1']/ df_point['m0'], ax=ax[1], legend=False, marker='o', ms=10, color=pal[0]) 
    sns.lineplot(end_point, x=idx_on, y=df_point_on['m1'] / df_point_on['m0'], ax=ax[1], legend=False, marker='o', ms=10, color=pal[1])
    
    ax[1].set_ylabel('$\mathcal{F}_1 / \mathcal{F}_0$')
    ax[1].set_xlabel('FF Input (Hz)')
    # ax[1].set_ylim([0., 1.5])
    
    point = df_smooth[df_smooth.Iext==idx_off].reset_index()
    m0, m1, phase = decode_bump(point.rates[0])
    point = np.roll(point.rates[0], int(( phase / 2.0 / np.pi - 0.5) * point.rates[0].shape[0]))
    
    point_on = df_smooth[df_smooth.Iext==idx_on].reset_index()  
    m0, m1, phase = decode_bump(point_on.rates[0])
    point_on = np.roll(point_on.rates[0], int((phase / 2.0 / np.pi - 0.5) * point_on.rates[0].shape[0]))
    
    ax[2].plot(point, color=pal[0])
    ax[2].plot(point_on, color=pal[1])
    
    ax[2].set_xticks([0, N_E/4, N_E/2, 3*N_E/4, N_E], [0, 90, 180, 270, 360])
    ax[2].set_ylabel('Firing Rate (Hz)')
    ax[2].set_xlabel('Pref. Location (°)')
    
    plt.savefig(name + '_tuning.svg', dpi=300)
    
    plt.show()

![img](./.ob-jupyter/306e00e58c47638734f39f29cedf478c325eeb42.png)



###### Diffusion



    point = end_point[end_point.Iext==idx_off]
    point_on = end_point[end_point.Iext==idx_on]
    
    DURATION = 3.5
    
    fig, ax = plt.subplots(1, 3, figsize=[3*width, height])
    
    sns.lineplot(end_point, x='Iext', y=end_point.precision**2 / DURATION , legend=False, ax=ax[0])
    
    sns.lineplot(x=idx_off, y=point['precision']**2 / DURATION, legend=False, marker='o', ax=ax[0], ms=10, color=pal[0])
    sns.lineplot(x=idx_on, y=point_on['precision']**2 / DURATION, legend=False, marker='o', ax=ax[0], ms=10, color=pal[1])
    
    ax[0].set_xlabel('FF Input ($mS/cm^2$)')
    ax[0].set_ylabel('Diffusivity ($\\text{deg}^2$/s)', color=sns.color_palette('deep')[0])
    ax[0].set_ylim([0, 40])
    
    ax1 = ax[0].twinx()
    sns.lineplot(end_point, x='Iext', y=end_point['m1']/end_point['m0'], ax=ax1, legend=False, color='k', alpha=0.5)
    
    sns.lineplot(end_point, x=idx_off, y=point['m1']/point['m0'], legend=False, marker='o', ax=ax1, ms=10, color=pal[0])
    sns.lineplot(end_point, x=idx_on, y=point_on['m1']/point_on['m0'], legend=False, marker='o', ax=ax1, ms=10, color=pal[1])
    
    ax1.set_ylabel('Amplitude, $\mathcal{F}_1 / \mathcal{F}_0$', alpha=0.5)
    ax1.spines['right'].set_visible(True)
    ax1.set_ylim([0., 1.25])
    ax1.set_yticks([0.0, 0.5, 1.0])
    
    plot_smooth(smooth, ax=ax[1], color=pal[0])
    plot_smooth(smooth_on, ax=ax[1], color=pal[1])
    
    ax[1].set_xticks([0, N_E/2, N_E], [0, 180, 360])
    ax[1].set_ylabel('Firing Rate (Hz)')
    ax[1].set_xlabel('Pref. Location (°)')
    ax[1].set_ylim([0, 15])
    
    bins = 'auto'
    sns.histplot(data=point, x=point['precision'], legend=False, ax=ax[2], bins=bins, kde=True, stat='density', element='step', alpha=0,color = pal[0])  
    sns.histplot(data=point_on, x=point_on['precision'], legend=False, ax=ax[2], bins=bins, kde=True, stat='density', element='step', alpha=0., color=pal[1])
    ax[2].set_xlabel('Endpoint Deviation (°)')
    ax[2].set_ylabel('Density')
    ax[2].set_yticks([0, 0.04, 0.08])
    ax[2].set_ylim([0, 0.08])
    ax[2].set_xlim([-20, 20])
    
    plt.savefig(name + '_diffusion.svg', dpi=300)
    plt.show()

![img](./.ob-jupyter/8132309c210f21b994abd0cdb466801d6e71e96e.png)



### Varying Jee



#### Parameters



    name = 'odr_Jee'
    conf_name = 'config'
    
    Jab_list = np.linspace(4, 4.5, 11)
    
    print(Jab_list)
    ini_list = np.arange(0, 1)
    phi_list = np.linspace(0, 315, 8)
    # ini_list = [0]
    # phi_list = [180]

    [4.   4.05 4.1  4.15 4.2  4.25 4.3  4.35 4.4  4.45 4.5 ]



#### Simulation



    run_loop_X(conf_name, name, 'Jab', Jab_list, ini_list, phi_list, axis=0)

    MEM_USAGE > 85.0%, sleeping for a while ...



#### Analysis



###### Load Data



    df = load_data(name, Jab_list, ini_list, phi_list, 'Jab')
    df_code, end_point = get_df_code(df, 'Jab')
    
    # end_point['accuracy'] = (end_point.phase - end_point['phi'] / 180 * np.pi) % (2 * np.pi)
    end_point['precision'] = end_point.groupby(['phi', 'Jab'], group_keys=False)['phase'].apply(get_precision) * 180 / np.pi
    df_smooth = df.groupby(['time', 'trial', 'phi', 'Jab'])['rates'].apply(circcvl).reset_index()

    (88, 32000)
    (1, 8, 11, 32000, 1)
       trial  phi   Jab        m0        m1     phase
    0      0    0  4.00  0.642375  0.031713 -0.807716
    1      0    0  4.05  0.671000  0.118463  2.856525
    2      0    0  4.10  0.664125  0.057427  0.658429
    3      0    0  4.15  0.681500  0.046423 -0.025012
    4      0    0  4.20  0.712000  0.030495 -0.101156

    idx_off = Jab_list[3]
    idx_on = Jab_list[8]
    print('parameters', idx_off, idx_on)

    parameters 4.15 4.4

    smooth=[]
    point = df_smooth[df_smooth.Jab==idx_off].reset_index()  
    for i in range(point.rates.shape[0]):
        m0, m1, phase = decode_bump(point.rates[i])
        smooth.append(np.roll(point.rates[i], int(( phase / 2.0 / np.pi - 0.5) * point.rates[i].shape[0])))
    
    smooth = np.array(smooth)
    
    smooth_on = []
    point_on = df_smooth[df_smooth.Jab==idx_on].reset_index()  
    for i in range(point_on.rates.shape[0]):
        m0, m1, phase = decode_bump(point_on.rates[i])
        smooth_on.append(np.roll(point_on.rates[i], int((phase / 2.0 / np.pi - 0.5) * point_on.rates[i].shape[0])))
    
    smooth_on = np.array(smooth_on)



###### Tuning



    N_E = 32000
    
    df_point = end_point[end_point.Jab==idx_off]
    df_point_on = end_point[end_point.Jab==idx_on]
    
    fig, ax = plt.subplots(1, 3, figsize=[2.25*width, height])
    
    sns.lineplot(end_point, x='Jab', y='m0', ax=ax[0], legend=False, hue='phi', lw=0, marker='o')
    ax[0].set_xlabel('$J_{EE}$')
    ax[0].set_ylabel('$\mathcal{F}_0$ (Hz)')
    
    ax1 = ax[0].twinx()
    sns.lineplot(end_point, x='Jab', y='m1', ax=ax1, legend=False, color='k', alpha=0.5, marker='o')
    
    # sns.lineplot(end_point, x=idx_off, y=point['m1'], legend=False, marker='o', ax=ax1, ms=10, color=pal[0])
    # sns.lineplot(end_point, x=idx_on, y=point_on['m1'], legend=False, marker='o', ax=ax1, ms=10, color=pal[1])
    
    ax1.set_ylabel('$\mathcal{F}_1 (Hz)$')
    ax1.spines['right'].set_visible(True)
    
    sns.lineplot(end_point, x='Jab', y=end_point['m1']/end_point['m0'], ax=ax[1], legend=False, marker='o', color='k')
    sns.lineplot(end_point, x=idx_off, y=df_point['m1']/ df_point['m0'], ax=ax[1], legend=False, marker='o', ms=10, color=pal[0]) 
    sns.lineplot(end_point, x=idx_on, y=df_point_on['m1'] / df_point_on['m0'], ax=ax[1], legend=False, marker='o', ms=10, color=pal[1])
    
    ax[1].set_ylabel('$\mathcal{F}_1 / \mathcal{F}_0$')
    ax[1].set_xlabel('$J_{EE}$')
    # ax[0].set_ylim([0.4, 1])
    
    point = df_smooth[df_smooth.Jab==idx_off].reset_index() 
    m0, m1, phase = decode_bump(point.rates[0])
    point = np.roll(point.rates[0], int(( phase / 2.0 / np.pi - 0.5) * point.rates[0].shape[0]))
    
    point_on = df_smooth[df_smooth.Jab==idx_on].reset_index()  
    m0, m1, phase = decode_bump(point_on.rates[0])
    point_on = np.roll(point_on.rates[0], int((phase / 2.0 / np.pi - 0.5) * point_on.rates[0].shape[0]))
    
    ax[2].plot(point, color=pal[0])
    ax[2].plot(point_on, color=pal[1])
    
    ax[2].set_xticks([0, N_E/4, N_E/2, 3*N_E/4, N_E], [0, 90, 180, 270, 360])
    ax[2].set_ylabel('Firing Rate (Hz)')
    ax[2].set_xlabel('Pref. Location (°)')
    
    plt.savefig(name + '_tuning.svg', dpi=300)
    
    plt.show()

![img](./.ob-jupyter/94f1bae666cd9309f5038d844f6dacc24a43c714.png)



###### Diffusion



    point = end_point[end_point.Jab==idx_off]
    point_on = end_point[end_point.Jab==idx_on]
    
    fig, ax = plt.subplots(1, 3, figsize=[3*width, height])
    
    sns.lineplot(end_point, x='Jab', y=end_point.precision**2 / 3.5 , legend=False, ax=ax[0])
    
    sns.lineplot(x=idx_off, y=point['precision']**2 / 3.5, legend=False, marker='o', ax=ax[0], ms=10, color=pal[0])
    sns.lineplot(x=idx_on, y=point_on['precision']**2 / 3.5, legend=False, marker='o', ax=ax[0], ms=10, color=pal[1])
    
    ax[0].set_xlabel('$J_{EE}$')
    ax[0].set_ylabel('Diffusivity ($\\text{deg}^2$/s)')
    ax[0].set_ylim([10, 50])
    
    ax1 = ax[0].twinx()
    sns.lineplot(end_point, x='Jab', y=end_point['m1']/end_point['m0'], ax=ax1, legend=False, color='k', alpha=0.5)
    
    sns.lineplot(end_point, x=idx_off, y=point['m1']/point['m0'], legend=False, marker='o', ax=ax1, ms=10, color=pal[0])
    sns.lineplot(end_point, x=idx_on, y=point_on['m1']/point_on['m0'], legend=False, marker='o', ax=ax1, ms=10, color=pal[1])
    
    ax1.set_ylabel('Amplitude, $\mathcal{F}_1 / \mathcal{F}_0$')
    ax1.spines['right'].set_visible(True)
    
    plot_smooth(smooth, ax=ax[1], color=pal[0])
    plot_smooth(smooth_on, ax=ax[1], color=pal[1])
    
    ax[1].set_xticks([0, N_E/4, N_E/2, 3*N_E/4, N_E], [0, 90, 180, 270, 360])
    ax[1].set_ylabel('Firing Rate (Hz)')
    ax[1].set_xlabel('Pref. Location (°)')
    
    bins = 'auto'
    sns.histplot(data=point, x=point['precision'], legend=False, ax=ax[2], bins=bins, kde=True, stat='density', element='step', alpha=0,color = pal[0])  
    sns.histplot(data=point_on, x=point_on['precision'], legend=False, ax=ax[2], bins=bins, kde=True, stat='density', element='step', alpha=0., color=pal[1])
    ax[2].set_xlabel('Endpoint Deviation (°)')
    ax[2].set_ylabel('Density')
    ax[2].set_xlim([-30, 30])
    
    plt.savefig(name + '_diffusion.svg', dpi=300)
    plt.show()

![img](./.ob-jupyter/eebc317cf407e3968040a726160b3d9dd3491b83.png)



### Varying M0



#### Parameters



    name = 'odr_last'
    conf_name = 'config'
    M0_list = np.linspace(0.75, 1.5, 11)
    print(M0_list)
    
    ini_list = np.arange(0, 10)
    phi_list = np.linspace(0, 315, 8)
    
    # ini_list = [0]
    # phi_list = [180]

    [0.75  0.825 0.9   0.975 1.05  1.125 1.2   1.275 1.35  1.425 1.5  ]



#### Simulation



    run_loop_X(conf_name, name, 'M0', M0_list, ini_list, phi_list, axis=None)

    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...



#### Analysis



###### Load Data



    df = load_data(name, M0_list, ini_list, phi_list, 'M0')
    df_code, end_point = get_df_code(df, 'M0')
    
    end_point['precision'] = end_point.groupby(['phi', 'M0'], group_keys=False)['phase'].apply(get_precision) * 180 / np.pi
    
    df_smooth = df.groupby(['time', 'trial', 'phi', 'M0'])['rates'].apply(circcvl).reset_index()

    (880, 32000)
    (10, 8, 11, 32000, 1)
       trial  phi     M0        m0        m1     phase
    0      0    0  0.750  0.798125  0.406200  0.069975
    1      0    0  0.825  1.633125  1.091774  0.666848
    2      0    0  0.900  3.108750  2.462080 -0.028826
    3      0    0  0.975  3.881875  2.309621 -0.059571
    4      0    0  1.050  4.966250  3.305753  0.207346



###### Tuning Profile



    idx_off = np.round(M0_list[2], 5)
    idx_on = np.round(M0_list[5], 5)
    print('parameters', idx_off, idx_on)

    parameters 0.9 1.125

    N_E = 32000
    
    df_point = end_point[end_point.M0==idx_off]
    df_point_on = end_point[end_point.M0==idx_on]
    
    fig, ax = plt.subplots(1, 3, figsize=[2.25*width, height])
    
    sns.lineplot(end_point, x='M0', y='m0', ax=ax[0], legend=False, marker='o', lw=0, hue='trial')
    ax[0].set_xlabel('FF Input')
    ax[0].set_ylabel('$\mathcal{F}_0$ (Hz)')
    
    sns.lineplot(end_point, x='M0', y=end_point['m1']/end_point['m0'], ax=ax[1], legend=False, marker='o', color='k')
    sns.lineplot(end_point, x=idx_off, y=df_point['m1']/ df_point['m0'], ax=ax[1], legend=False, marker='o', ms=10, color=pal[0]) 
    sns.lineplot(end_point, x=idx_on, y=df_point_on['m1'] / df_point_on['m0'], ax=ax[1], legend=False, marker='o', ms=10, color=pal[1])
    
    ax[1].set_ylabel('$\mathcal{F}_1 / \mathcal{F}_0$')
    ax[1].set_xlabel('FF Input (Hz)')
    # ax[0].set_ylim([0.4, 1])
    
    smooth=[]
    point = df_smooth[df_smooth.M0==idx_off].reset_index()  
    for i in range(point.rates.shape[0]):
        m0, m1, phase = decode_bump(point.rates[i])
        smooth.append(np.roll(point.rates[i], int(( phase / 2.0 / np.pi - 0.5) * point.rates[i].shape[0])))
    
    smooth_on = []
    point_on = df_smooth[df_smooth.M0==idx_on].reset_index()  
    for i in range(point_on.rates.shape[0]):
        m0, m1, phase = decode_bump(point_on.rates[i])
        smooth_on.append(np.roll(point_on.rates[i], int((phase / 2.0 / np.pi - 0.5) * point_on.rates[i].shape[0])))
    
    ax[2].plot(np.mean(smooth, 0), color=pal[0])
    ax[2].plot(np.mean(smooth_on, 0), color=pal[1])
    
    ax[2].set_xticks([0, N_E/4, N_E/2, 3*N_E/4, N_E], [0, 90, 180, 270, 360])
    ax[2].set_ylabel('Firing Rate (Hz)')
    ax[2].set_xlabel('Pref. Location (°)')
    
    plt.savefig(name + '_tuning.svg', dpi=300)
    
    plt.show()

![img](./.ob-jupyter/a8cdfdaca91d812959b566676aeb3913c3d681a4.png)



###### Diffusion



    point = end_point[end_point.M0==idx_off]
    point_on = end_point[end_point.M0==idx_on]
    
    fig, ax = plt.subplots(1, 3, figsize=[3*width, height])
    
    sns.lineplot(end_point, x='M0', y=end_point.precision**2 / 3.5 , legend=False, ax=ax[0])
    
    sns.lineplot(x=idx_off, y=point['precision']**2 / 3.5, legend=False, marker='o', ax=ax[0], ms=10, color=pal[0])
    sns.lineplot(x=idx_on, y=point_on['precision']**2 / 3.5, legend=False, marker='o', ax=ax[0], ms=10, color=pal[1])
    
    ax[0].set_xlabel('FF Input (Hz)')
    ax[0].set_ylabel('Diffusivity ($\\text{deg}^2$/s)')
    ax[0].set_ylim([10, 60])
    
    ax1 = ax[0].twinx()
    sns.lineplot(end_point, x='M0', y=end_point['m1']/end_point['m0'], ax=ax1, legend=False, color='k', alpha=0.5)
    
    sns.lineplot(end_point, x=idx_off, y=point['m1']/point['m0'], legend=False, marker='o', ax=ax1, ms=10, color=pal[0])
    sns.lineplot(end_point, x=idx_on, y=point_on['m1']/point_on['m0'], legend=False, marker='o', ax=ax1, ms=10, color=pal[1])
    
    ax1.set_ylabel('Amplitude, $\mathcal{F}_1 / \mathcal{F}_0$')
    ax1.spines['right'].set_visible(True)
    
    ax[1].plot(np.mean(smooth, 0), color=pal[0])
    ax[1].plot(np.mean(smooth_on, 0), color=pal[1])
    
    ax[1].set_xticks([0, N_E/4, N_E/2, 3*N_E/4, N_E], [0, 90, 180, 270, 360])
    ax[1].set_ylabel('Firing Rate (Hz)')
    ax[1].set_xlabel('Pref. Location (°)')
    
    bins = 'auto'
    sns.histplot(data=point, x=point['precision'], legend=False, ax=ax[2], bins=bins, kde=True, stat='density', element='step', alpha=0,color = pal[0])
    sns.histplot(data=point_on, x=point_on['precision'], legend=False, ax=ax[2], bins=bins, kde=True, stat='density', element='step', alpha=0., color=pal[1])
    ax[2].set_xlabel('Endpoint Deviation (°)')
    ax[2].set_ylabel('Density')
    # ax[1].set_xlim([-20, 20])
    
    plt.savefig(name + '_diffusion.svg', dpi=300)
    plt.show()

![img](./.ob-jupyter/79f0c5db0fae59bdf2a083eb4dd741f343702c48.png)



### Varying Tau fac



#### Parameters



    name = 'odr_tau'
    conf_name = 'config'
    TAU_FAC_list = np.linspace(500, 700, 11)
    print(TAU_FAC_list)
    
    ini_list = np.arange(0, 50)
    phi_list = np.linspace(0, 315, 8)
    # phi_list = [180]
    # ini_list = [0]

    [500. 520. 540. 560. 580. 600. 620. 640. 660. 680. 700.]



#### Simulation



    run_loop_X(conf_name, name, 'TAU_FAC', TAU_FAC_list, ini_list, phi_list, axis=0)

    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...
    MEM_USAGE > 85.0%, sleeping for a while ...



#### Analysis



###### Load Data



    df = load_data(name, TAU_FAC_list, ini_list, phi_list, 'TAU_FAC')
    df_code, end_point = get_df_code(df, 'TAU_FAC')
    
    end_point['accuracy'] = (end_point.phase - end_point['phi'] / 180 * np.pi) % (2 * np.pi)
    end_point['precision'] = end_point.groupby(['phi', 'TAU_FAC'], group_keys=False)['phase'].apply(get_precision) * 180 / np.pi
    
    df_smooth = df.groupby(['time', 'trial', 'phi', 'TAU_FAC'])['rates'].apply(circcvl).reset_index()

    (4400, 32000)
    (50, 8, 11, 32000, 1)
       trial  phi  TAU_FAC        m0        m1     phase
    0      0    0    500.0  2.270000  0.743045 -0.124722
    1      0    0    520.0  3.221250  1.846122  0.179287
    2      0    0    540.0  3.915625  2.483390  0.254242
    3      0    0    560.0  4.370625  2.714665 -0.099200
    4      0    0    580.0  5.070625  3.233977  0.004786

    idx_off = TAU_FAC_list[4]
    idx_on = TAU_FAC_list[-1]
    print('parameters', idx_off, idx_on)

    parameters 580.0 700.0

    smooth=[]
    point = df_smooth[df_smooth.TAU_FAC==idx_off].reset_index()  
    for i in range(point.rates.shape[0]):
        m0, m1, phase = decode_bump(point.rates[i])
        smooth.append(np.roll(point.rates[i], int(( phase / 2.0 / np.pi - 0.5) * point.rates[i].shape[0])))
    
    smooth = np.array(smooth)
    
    smooth_on = []
    point_on = df_smooth[df_smooth.TAU_FAC==idx_on].reset_index()  
    for i in range(point_on.rates.shape[0]):
        m0, m1, phase = decode_bump(point_on.rates[i])
        smooth_on.append(np.roll(point_on.rates[i], int((phase / 2.0 / np.pi - 0.5) * point_on.rates[i].shape[0])))
    
    smooth_on = np.array(smooth_on)



###### Tuning Profile



    N_E = 32000
    
    df_point = end_point[end_point.TAU_FAC==idx_off]
    df_point_on = end_point[end_point.TAU_FAC==idx_on]
    
    fig, ax = plt.subplots(1, 3, figsize=[2.25*width, height])
    
    sns.lineplot(end_point, x='TAU_FAC', y='m0', ax=ax[0], legend=False, marker='o', lw=0, hue='trial')
    ax[0].set_xlabel('FF Input')
    ax[0].set_ylabel('$\mathcal{F}_0$ (Hz)')
    ax[0].set_ylim([0, 20])
    
    sns.lineplot(end_point, x='TAU_FAC', y=end_point['m1']/end_point['m0'], ax=ax[1], legend=False, marker='o', color='k')
    sns.lineplot(end_point, x=idx_off, y=df_point['m1']/ df_point['m0'], ax=ax[1], legend=False, marker='o', ms=10, color=pal[0]) 
    sns.lineplot(end_point, x=idx_on, y=df_point_on['m1'] / df_point_on['m0'], ax=ax[1], legend=False, marker='o', ms=10, color=pal[1])
    
    ax[1].set_ylabel('$\mathcal{F}_1 / \mathcal{F}_0$')
    ax[1].set_xlabel('$\\tau_{fac}$ (ms)')
    # ax[0].set_ylim([0.4, 1])
    
    
    point = df_smooth[df_smooth.TAU_FAC==idx_off].reset_index() 
    m0, m1, phase = decode_bump(point.rates[0])
    point = np.roll(point.rates[0], int(( phase / 2.0 / np.pi - 0.5) * point.rates[0].shape[0]))
    
    point_on = df_smooth[df_smooth.TAU_FAC==idx_on].reset_index()  
    m0, m1, phase = decode_bump(point_on.rates[0])
    point_on = np.roll(point_on.rates[0], int((phase / 2.0 / np.pi - 0.5) * point_on.rates[0].shape[0]))
    
    ax[2].plot(point, color=pal[0])
    ax[2].plot(point_on, color=pal[1])
    
    ax[2].set_xticks([0, N_E/4, N_E/2, 3*N_E/4, N_E], [0, 90, 180, 270, 360])
    ax[2].set_ylabel('Firing Rate (Hz)')
    ax[2].set_xlabel('Pref. Location (°)')
    
    plt.savefig(name + '_tuning.svg', dpi=300)
    
    plt.show()

![img](./.ob-jupyter/ae6f9046b3f40b9a0eaa24520bc844eaa811e233.png)



###### Diffusion



    point = end_point[end_point.TAU_FAC==idx_off]
    point_on = end_point[end_point.TAU_FAC==idx_on]
    
    fig, ax = plt.subplots(1, 3, figsize=[3*width, height])
    
    sns.lineplot(end_point, x='TAU_FAC', y=end_point.precision**2 / 3.5 , legend=False, ax=ax[0])
    
    sns.lineplot(x=idx_off, y=point['precision']**2 / 3.5, legend=False, marker='o', ax=ax[0], ms=10, color=pal[0])
    sns.lineplot(x=idx_on, y=point_on['precision']**2 / 3.5, legend=False, marker='o', ax=ax[0], ms=10, color=pal[1])
    
    ax[0].set_xlabel('$\\tau_{fac}$ (ms)')
    ax[0].set_ylabel('Diffusivity ($\\text{deg}^2$/s)')
    ax[0].set_ylim([15, 50])
    
    ax1 = ax[0].twinx()
    sns.lineplot(end_point, x='TAU_FAC', y=end_point['m1']/end_point['m0'], ax=ax1, legend=False, color='k', alpha=0.25)
    
    sns.lineplot(end_point, x=idx_off, y=point['m1']/point['m0'], legend=False, marker='o', ax=ax1, ms=10, color=pal[0])
    sns.lineplot(end_point, x=idx_on, y=point_on['m1']/point_on['m0'], legend=False, marker='o', ax=ax1, ms=10, color=pal[1])
    
    ax1.set_ylabel('Amplitude, $\mathcal{F}_1 / \mathcal{F}_0$')
    ax1.spines['right'].set_visible(True)
    ax1.set_ylim([0.25, 0.75])
    
    plot_smooth(smooth, ax=ax[1], color=pal[0])
    plot_smooth(smooth_on, ax=ax[1], color=pal[1])
    
    ax[1].set_xticks([0, N_E/4, N_E/2, 3*N_E/4, N_E], [0, 90, 180, 270, 360])
    ax[1].set_ylabel('Firing Rate (Hz)')
    ax[1].set_xlabel('Pref. Location (°)')
    ax[1].set_ylim([0, 13])
    ax[1].set_yticks([0, 5 , 10])
    
    bins = 'auto'
    sns.histplot(data=point, x=point['precision'], legend=False, ax=ax[2], bins=bins, kde=True, stat='density', element='step', alpha=0,color = pal[0])  
    sns.histplot(data=point_on, x=point_on['precision'], legend=False, ax=ax[2], bins=bins, kde=True, stat='density', element='step', alpha=0., color=pal[1])
    ax[2].set_xlabel('Endpoint Deviation (°)')
    ax[2].set_ylabel('Density')
    # ax[2].set_xlim([-30, 30])
    
    plt.savefig(name + '_diffusion.svg', dpi=300)
    plt.show()

![img](./.ob-jupyter/e6387fee8f535811760eff07847d8af3220e8f69.png)

